In [2]:
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Utils as utils
import seaborn as sns
import warnings
from sklearn import ensemble
from sklearn import multioutput
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler,RobustScaler

c:\Users\HAMZA\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [6]:
train_df = pd.read_parquet("train_final.parquet")
test = pd.read_parquet("test_final.parquet")

In [7]:
def do_Target_spareted(dataframe):
    new_spareted_cabin = dataframe["target"].str.split(pat = ",", expand = True)
    
    dataframe.drop("target", axis=1, inplace=True)
    
    new_spareted_cabin.rename(columns={0 : 'first_menu',
                                       1 : 'second_menu',
                                       2 : 'third_menu'}, inplace=True)
    
    return pd.concat([dataframe, new_spareted_cabin], axis=1)

In [8]:
train_df = do_Target_spareted(train_df)

In [9]:
lb = LabelBinarizer()
transformed_data_first = lb.fit_transform(train_df["first_menu"])
transformed_data_second = lb.fit_transform(train_df["second_menu"])
transformed_data_third = lb.fit_transform(train_df["third_menu"])

In [10]:
transformed_data_second = [np.insert(row, 4, 0) for row in transformed_data_second]
transformed_data_third = [np.insert(row, 2, 0) for row in transformed_data_third]

In [11]:
transformed_data_list = [row.tolist() for row in transformed_data_first]

# Listeyi Pandas Serisi'ne dönüştürme
transformed_data_series = pd.Series(transformed_data_list)


transformed_data_list_second = [row.tolist() for row in transformed_data_second]

# Listeyi Pandas Serisi'ne dönüştürme
transformed_data_series_second = pd.Series(transformed_data_list_second)


transformed_data_list_third = [row.tolist() for row in transformed_data_third]

# Listeyi Pandas Serisi'ne dönüştürme
transformed_data_series_third = pd.Series(transformed_data_list_third)

In [12]:
train_df["first_menu"] = transformed_data_series
train_df["second_menu"] = transformed_data_series_second
train_df["third_menu"] = transformed_data_series_third

In [13]:
train_df['target'] = train_df.apply(lambda row: [1 if any(x) else 0 for x in zip(*row[['first_menu', 'second_menu', 'third_menu']])], axis=1)

In [14]:
train_df.drop(["first_menu","second_menu","third_menu"], inplace=True, axis=1)

In [15]:
X = train_df.drop("target", axis=1)
y = train_df["target"]

In [16]:
numpy_dizi = y.to_numpy()
numpy_dizi = [np.array(row) for row in numpy_dizi]

In [17]:
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'VODAFONE' if 'VODAFONE' in x else x)
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'VODAFONE' if 'VF' in x else x)
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'TURKCELL' if 'LIFECELL' in x else x)
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TURK TELEKO' in x else x)
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TÜRK TELEKO' in x else x)
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TURKTELEKOM' in x else x)
train_df['carrier'] = train_df['carrier'].str.replace('^KCELL ', 'AKCELL', regex=True) 

yurtdisi = ['ALMADAR','AIRTEL','AZERCELL','BAKCELL','O2','BEE','A1 ','3_AT','IRANCELL',
            'AYYILDIZ','BH','NL','ORANGE','MOLDCELL','ZAIN','YETTEL','VERIZONE','TELEKOM',
            'TELENOR','TELE2','TELIA','MAGTI','STC','BOUYGUES','HORMUUD','JIO','LIDL','KSA',
            'FREEDOM','BUDGET','XFINITY','CHINA_TELECOM','MTN','1&1','BASE','CLARO','GEOCELL',
            'MEGAFONE','GSMOBILE','ETISALAT','TIM','MAXIS','PROXIMUS', 'SUNRISE', 'WINDTRE', 
            'VODACOM', 'LYCAMOBILE','LIBYANA','TIGO', 'ASIACELL', 'SFR','CUBACELL','AKCELL','SALT','T-MOBILE'] 
sirket = ['TURKCELL','BIMCELL','AVEA','VODAFONE','TURK_TELEKOM','TEKNOSA','PTTCELL','KKTCELL']

birlesim = yurtdisi+sirket

for anahtarKelime in birlesim:
    train_df['carrier'] = train_df['carrier'].apply(lambda x: anahtarKelime if anahtarKelime in x else x)
    
unknown1 = ['HAYAT','FENER','TRABZON','61','UNKNOWN',' ','nknown']
for anahtarKelime in unknown1:
    train_df['carrier'] = train_df['carrier'].apply(lambda x: 'UNKNOWN' if anahtarKelime in x else x)
    
b=train_df['carrier'].unique()

unknown = [row for row in b if row not in birlesim]
train_df['carrier'] = train_df['carrier'].apply(lambda x: 'UNKNOWN' if x in unknown else x if x not in unknown else x)

In [18]:
ordinal = ['carrier', 'devicebrand'] 
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
train_df[ordinal] = ordinal_encoder.fit_transform(train_df[ordinal]) 

In [19]:
test['carrier'] = test['carrier'].apply(lambda x: 'VODAFONE' if 'VODAFONE' in x else x)
test['carrier'] = test['carrier'].apply(lambda x: 'VODAFONE' if 'VF' in x else x)
test['carrier'] = test['carrier'].apply(lambda x: 'TURKCELL' if 'LIFECELL' in x else x)
test['carrier'] = test['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TURK TELEKO' in x else x)
test['carrier'] = test['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TÜRK TELEKO' in x else x)
test['carrier'] = test['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TURKTELEKOM' in x else x)
test['carrier'] = test['carrier'].str.replace('^KCELL ', 'AKCELL', regex=True) 

yurtdisi = ['ALMADAR','AIRTEL','AZERCELL','BAKCELL','O2','BEE','A1 ','3_AT','IRANCELL',
            'AYYILDIZ','BH','NL','ORANGE','MOLDCELL','ZAIN','YETTEL','VERIZONE','TELEKOM',
            'TELENOR','TELE2','TELIA','MAGTI','STC','BOUYGUES','HORMUUD','JIO','LIDL','KSA',
            'FREEDOM','BUDGET','XFINITY','CHINA_TELECOM','MTN','1&1','BASE','CLARO','GEOCELL',
            'MEGAFONE','GSMOBILE','ETISALAT','TIM','MAXIS','PROXIMUS', 'SUNRISE', 'WINDTRE', 
            'VODACOM', 'LYCAMOBILE','LIBYANA','TIGO', 'ASIACELL', 'SFR','CUBACELL','AKCELL','SALT','T-MOBILE'] 
sirket = ['TURKCELL','BIMCELL','AVEA','VODAFONE','TURK_TELEKOM','TEKNOSA','PTTCELL','KKTCELL']

birlesim = yurtdisi+sirket

for anahtarKelime in birlesim:
    test['carrier'] = test['carrier'].apply(lambda x: anahtarKelime if anahtarKelime in x else x)
    
unknown1 = ['HAYAT','FENER','TRABZON','61','UNKNOWN',' ','nknown']
for anahtarKelime in unknown1:
    test['carrier'] = test['carrier'].apply(lambda x: 'UNKNOWN' if anahtarKelime in x else x)
    
b=test['carrier'].unique()

unknown = [row for row in b if row not in birlesim]
test['carrier'] = test['carrier'].apply(lambda x: 'UNKNOWN' if x in unknown else x if x not in unknown else x)

In [20]:
ordinal = ['carrier', 'devicebrand'] 
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
test[ordinal] = ordinal_encoder.fit_transform(test[ordinal]) 

In [21]:
test.drop("id", axis=1,inplace=True)

In [22]:
X = train_df.drop("target", axis=1)
y = train_df["target"]
X = X.drop(["id"], axis=1)

In [19]:
reg = multioutput.MultiOutputRegressor(ensemble.GradientBoostingRegressor())
reg.fit(X, numpy_dizi)

MultiOutputRegressor(estimator=GradientBoostingRegressor())

In [20]:
y_pred = reg.predict(test)

In [21]:
y_pred

array([[ 0.24345104,  0.78853258,  0.09088579, ...,  0.31445502,
         0.28746954,  0.24959528],
       [ 0.32557741,  0.57686075,  0.01418683, ...,  0.10925412,
         0.05733983,  0.25882473],
       [ 0.12069751,  0.87852954,  0.02485349, ...,  0.04852757,
         0.50638738,  0.12885674],
       ...,
       [ 0.17974986,  0.75966793,  0.11488464, ...,  0.09703592,
        -0.00848688,  0.27685238],
       [ 0.18230759,  0.75539181,  0.22073843, ...,  0.05101418,
         0.39024897,  0.23389084],
       [ 0.26847778,  0.64368436,  0.264168  , ...,  0.04970338,
         0.00336058,  0.26756072]])

In [143]:
def sample_sub(ypred):
    

    sample = pd.read_csv("csv_sample.csv")

    submission = pd.DataFrame({"id": sample["id"],
                                "target": ypred})

    # submission["target"] = submission["target"].astype(int)
                                
    submission.to_csv("01.csv", index=False)

In [23]:
import joblib
model = joblib.load('model_dosyasi.pkl')

In [144]:
y_pred = model.predict(test)

In [113]:
type(y_pred)

numpy.ndarray

In [145]:
for row in y_pred:
    list_with_index = [(value, index) for index, value in enumerate(row)]
    sorted_list = sorted(list_with_index, key=lambda x: x[0])
    three_values = [x[1] for x in sorted_list[-3:]]
    row[three_values]=1
    row[~np.isin(np.arange(len(row)), three_values)] = 0
    

In [115]:
type(row)

numpy.ndarray

In [146]:
y_pred = y_pred.astype(int)

In [140]:
y_pred

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [147]:
y_pred = [row.tolist() for row in y_pred]
# y_pred = pd.Series(y_pred)

sample_sub(pd.Series(y_pred))

In [109]:
zero = pd.read_csv('01.csv')
utils.check_df(zero)

##################### Shape #####################
(11955, 2)
##################### Types #####################
id        object
target    object
dtype: object
##################### Head #####################
                          id                       target
0  2e6105f5911256f4f6c4813ed  [0, 1, 0, 1, 0, 1, 0, 0, 0]
1  c56ad71dae0a5dbd3e7d36adc  [0, 1, 0, 1, 0, 1, 0, 0, 0]
2  4d02ea175f6581f0c6385311f  [0, 1, 0, 0, 0, 1, 0, 1, 0]
3  3412d27a86c286ba078fa935c  [0, 1, 0, 1, 0, 1, 0, 0, 0]
4  0203b561f6f7e10eafa46eefa  [0, 1, 0, 0, 0, 1, 0, 1, 0]
##################### Tail #####################
                              id                       target
11950  7687113f46112edf4f56666ee  [0, 1, 0, 1, 0, 0, 0, 0, 1]
11951  5ff8eb7a06fd48b60dbc04f34  [0, 1, 0, 1, 0, 1, 0, 0, 0]
11952  ac23a7b9ad3e5d61e738c854b  [0, 1, 0, 1, 0, 1, 0, 0, 0]
11953  7da05018634ea2eee4b122756  [0, 1, 0, 1, 0, 1, 0, 0, 0]
11954  57dbc6a230a851e6e2ee5c429  [0, 1, 0, 1, 0, 1, 0, 0, 0]
##################### N